# Metaファミリーモデルでの開発

## はじめに

このレッスンでは以下の内容を扱います：

- Metaファミリーの2つの主要モデル、Llama 3.1とLlama 3.2の紹介
- 各モデルの利用シーンや用途の理解
- 各モデルの特徴を示すコードサンプル

## Metaファミリーのモデル

このレッスンでは、Metaファミリー、または「Llama Herd」と呼ばれる2つのモデル、Llama 3.1とLlama 3.2を紹介します。

これらのモデルはさまざまなバリエーションがあり、Github Modelマーケットプレイスで利用できます。Github Modelsを使って[AIモデルでプロトタイピングする方法](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)について、さらに詳しく知ることができます。

モデルのバリエーション：
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*注：Llama 3もGithub Modelsで利用可能ですが、このレッスンでは扱いません*


## Llama 3.1

4050億パラメータを持つLlama 3.1は、オープンソースのLLMカテゴリに属します。

このモデルは、以前のLlama 3から以下の点でアップグレードされています：

- より大きなコンテキストウィンドウ - 128kトークン（従来は8kトークン）
- 最大出力トークン数の増加 - 4096（従来は2048）
- 多言語対応の向上 - 学習トークン数の増加による

これらの強化により、Llama 3.1はGenAIアプリケーションの構築時に、より複雑なユースケースに対応できるようになりました。主な例は以下の通りです：
- ネイティブ関数呼び出し - LLMのワークフロー外で外部ツールや関数を呼び出す機能
- RAGパフォーマンスの向上 - より広いコンテキストウィンドウによる
- 合成データ生成 - ファインチューニングなどのタスクに有効なデータを作成する能力


### ネイティブ関数呼び出し

Llama 3.1は、関数やツールの呼び出しをより効果的に行えるようにファインチューニングされています。また、ユーザーからのプロンプトに応じて、モデルが使用すべきだと判断できる2つの組み込みツールも備えています。これらのツールは以下の通りです。

- **Brave Search** - ウェブ検索を行うことで天気などの最新情報を取得できます
- **Wolfram Alpha** - より複雑な数学的計算に利用できるため、自分で関数を書く必要がありません

また、LLMが呼び出せる独自のカスタムツールを作成することも可能です。

以下のコード例では:

- 利用可能なツール（brave_search, wolfram_alpha）をシステムプロンプトで定義します。
- 特定の都市の天気について尋ねるユーザープロンプトを送信します。
- LLMはBrave Searchツールへのツールコールで応答します。これは次のような形になります `<|python_tag|>brave_search.call(query="Stockholm weather")`

*注: この例はツールコールのみを行います。結果を取得したい場合は、Brave APIページで無料アカウントを作成し、関数自体を定義する必要があります。


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Llama 3.1の制限の一つは、LLMでありながらマルチモーダル対応ができないことです。つまり、画像など異なる種類の入力をプロンプトとして使い、応答を返すことができません。この機能はLlama 3.2の主な特徴の一つです。その他の特徴は以下の通りです。

- マルチモーダル対応 - テキストと画像の両方のプロンプトを評価できる
- 小規模から中規模のバリエーション（11Bと90B） - 柔軟なデプロイメントが可能
- テキスト専用バリエーション（1Bと3B） - エッジやモバイルデバイスでの利用や低遅延を実現

マルチモーダル対応は、オープンソースモデルの世界において大きな進歩です。下記のコード例では、画像とテキストの両方のプロンプトを使い、Llama 3.2 90Bから画像の分析を得ています。

### Llama 3.2によるマルチモーダル対応


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## 学びはここで終わりではありません。旅を続けましょう

このレッスンを終えたら、[Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) をチェックして、生成AIの知識をさらに深めましょう！



---

**免責事項**:  
本書類は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性には努めておりますが、自動翻訳には誤りや不正確な表現が含まれる場合があります。原文（元の言語の文書）が正式な情報源と見なされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じたいかなる誤解や誤訳についても、当方は一切の責任を負いかねます。
